<img src="insight_logo.png" ALIGN=left style="margin: 0px 30px 30px 0px;" width="120"/> <font size="6"> Interview Demo: **overcast** </font> 

## Introduction

Just a few decades ago it was common for many astronomers to spend a significant fraction of their careers sitting behind the eyepiece of a telescope. Now the field is moving towards a model where most astronomers use data from a few billion-dollar telescopes in space or on near-perfect mountaintop sites that observe with more and more autonomy. 

One of the remaining crucial uses of (relatively) small-format telescopes is the rapid prototyping of new technologies. A flagship example is Dragonfly, shown below, a 1-meter class telescope built from an array of 48 commercial Canon telephoto lenses with specialized optical coatings.

<img src="dragonfly.jpg" ALIGN=left style="margin: 0px 10px 0px 0px;" width="300"/> 

Dragonfly has made many amazing discoveries but its team is just three professors and a handful of postdocs, graduate students, and undergraduates. One of them must be awake to observe (remotely) on the telescope every night, so they too have been moving towards a fully automated observing pipeline and have optimized for e.g. target selection as a function of position on sky and survey priority. 

A remaning roadblock to full autonomy - and a good night's sleep for all! - is the weather. Dragonfly has a good location but compared to that of billion-dollar class instruments the site's weather is quite variable. The observatory has an all-sky cloud sensor, meaning that stopping observations for overcast conditions could forego valueable data collection while a portion of the sky is clear. The site also has a webcam that the observer can use to modify the scheduling. Here we use machine learning to accomplish that task, building classification and prediction tools that the telescope's scheduler can query to adapt to the site conditions.

## Problem

1. Determine where a target lies on the webcam image
2. Decide whether that part of the image is cloudy
3. Predict if its status will change in the next ten minutes (Dragonfly's basic time unit)

## Data

<img src="2020-04-06/hires_04h12m22s_2020-04-07_EDT.jpg" ALIGN=left style="margin: 0px 10px 0px 0px;" width="400"/>  The epoch data are three images from the New Mexico Skies site website consisting of a high-resolution all-sky image, a low-resolution all-sky image, and a plot of the cloud sensor reading for the last few hours. The low-resoution image has only 1/10 as many pixels and tends to have more internal reflections but the high-resolution image seems less reliable, sometimes not changing over hours with seemingly no reason. We poll the website for these images every minute during the night. A typical high-res image is shown at left.

## Data access 

Spawn a subprocess to fetch the images and put them in a folder for today's date; this will run through the night

In [2]:
%matplotlib widget
import os
import sys
import time
import subprocess
import numpy as np
import matplotlib.pyplot as plt

In [3]:
today = time.strftime('%Y-%m-%d')
#Popen is non-blocking
#Toggle this statement - careful of opening a bunch of instances
if False: imgetter = subprocess.Popen(["python", "get_images.py", today])

In [ ]:
print(imgetter.returncode)
imgetter.kill()

## Matching pixels to sky positions

To decide whether a target is in clear sky we must first map sky positions to pixel coordinates. Typical astronomical images from e.g. the *Hubble Space Telescope* cover only a small area and so this transformation can be excellently approximated using the image tangent plane, one-to-one. However, it is apparent from inspection that the camera has a 'circular fisheye' lens - the image circle is inscribed within the sensor area, and there is a substantial (intentional) distortion from the usual mapping relations. 

Equisolid angle fisheyes are the most common comercially and have the useful property that each pixel covers the same area of sky. Here we assume this is the case for the New Mexico Skies fisheye; our results below lend confidence to this. For equisolid angle lenses, the mapping between the radius $r$ in pixels from the center of the frame and the angle $\theta$ from the optical axis is

$r = k_1 f \sin(\theta/k_2)$

where $k_{1}$ and $k_{2}$ depend on details of the lense shape and $f$ is a function of the sensor size and pixel scale.

Sky positions and other data about stars likely visible to the camera was obtained from the Yale Bright Star Catalog (Dorrit & Carlos 1991), which contains 9,095 stars and approximates all those visibile with the naked eye. We identified  a subset of bright stars in the high-resolution images for three recongiziable constellations - Scorpio, Ursa Major, and Lyra - to compare their pixel positions with those calculated using the catalog, location, and time of day.

In [3]:
from camera_mapping import star_pos_check
star_pos_check()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Above we have transformed the catalog positions to pixel coordinates using the equation above, assuming that the optical axis is pointed exactly at the zenith at that the top of the figure is pointed North, as specified by NMSkies. We take fiducial values $f=900$ and $k_1=k_2=2$. While the inferred positions (blue) are relatively close to the observed positions (red), clearly there is an important and nontrivial offset (not a rotation or translation) missing.

To identify the true mapping we use an affine-invariant Markov Chain Monte Carlo ensemble sampler called *emcee*, implemented in pure Python (Foreman-Mackey et al. 2013). We have six model parameters - $f,\ k_1,\ k_2$, the camera rotation East of North, and the pixel coordinates that represents the zenith. However, $f$ and $k_1$ are highly degenerate since they appear only multiplied with eachother, so we reduce the dimensinality by considering $f\times k_1 $, the 'scaling', instead. The likelihood function is the Euclidian distance between observed positions and mapped positions, and we assume wide Gaussian priors on all but $k_1$ and $k_2$, for which we use a uniform distribution between 1 and 3.

Below we show a corner plot representing the 1-d and 2-d marginalized likelihoods for each parameter.

<img src="corner_plot_2.png" style="margin: 10px 10px 10px 10px;" width="600"/> 

The parameters are extremely well constrained. The scaling and $k_2$ are significantly correlated but the fractional uncertainty in both is quite small. The zenith position is determined to less than a pixel and the rotation to a few hundredths of a degree. As shown below, the match between positions is immensely better (note that Lyra is shown at a larger relative scale). Further improvements will accouting for e.g. atmospheric refraction can produce only negligble progress towards the goal of cloudiness forecasts for a particular part of the sky.


In [4]:
star_pos_check(scalefac = 2141.4, rot = 4.58, k2=2.30, xc = 1969.5, yc = 1371.4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

## Data properties

In this section we examine the images to identify features to use for classification.

Unfortunately this week the moon is up which complicates things - this is normally considered suboptimal observing conditions but we will make do. The central 1600x1600 pixel area would be nice but here we use the top half of it to remove most of the Moon's influence.

In [5]:
cloudy_filenames=[
'2020-04-06/hires_00h01m58s_2020-04-07_EDT.jpg',
'2020-04-06/hires_00h06m08s_2020-04-07_EDT.jpg',
'2020-04-06/hires_00h11m20s_2020-04-07_EDT.jpg',
'2020-04-06/hires_00h18m35s_2020-04-07_EDT.jpg',
'2020-04-06/hires_00h24m49s_2020-04-07_EDT.jpg',
'2020-04-06/hires_04h06m08s_2020-04-07_EDT.jpg',
'2020-04-06/hires_04h12m22s_2020-04-07_EDT.jpg',
'2020-04-06/hires_04h17m34s_2020-04-07_EDT.jpg',
'2020-04-06/hires_04h22m47s_2020-04-07_EDT.jpg',
'2020-04-06/hires_04h28m00s_2020-04-07_EDT.jpg',
]

clear_filenames=[
'2020-04-02/hires_05h04m11s_2020-04-03_EDT.jpg',
'2020-04-02/hires_05h09m13s_2020-04-03_EDT.jpg',
'2020-04-02/hires_05h14m17s_2020-04-03_EDT.jpg',
'2020-04-02/hires_05h19m20s_2020-04-03_EDT.jpg',
'2020-04-02/hires_05h24m23s_2020-04-03_EDT.jpg',
]

#no moon
#clear_filenames=[
#'2020-04-03/hires_06h54m10s_2020-04-04_EDT.jpg',
#'2020-04-03/hires_07h00m33s_2020-04-04_EDT.jpg',
#'2020-04-03/hires_07h04m48s_2020-04-04_EDT.jpg',
#'2020-04-03/hires_07h10m09s_2020-04-04_EDT.jpg',
#'2020-04-03/hires_07h17m36s_2020-04-04_EDT.jpg',
#]

plt.figure()
for i in np.arange(4):
    plt.subplot(2,2,i+1)
    plt.imshow(plt.imread(clear_filenames[i])[536:2136,1204:2804,0], cmap='gray')
    plt.xticks([])
    plt.yticks([])
plt.suptitle('Clear')
plt.figure()
for i in np.arange(9):
    plt.subplot(3,3,i+1)
    plt.imshow(plt.imread(cloudy_filenames[i])[536:2136,1204:2804,0], vmax=170, cmap='gray')
    plt.xticks([])
    plt.yticks([])
plt.suptitle('Cloudy')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Text(0.5,0.98,'Cloudy')

We can also look at the pixel value histograms to get a better idea of the difference between the clear and cloudy images.

In [6]:
#plot histograms of pixel values
plt.figure()
plt.subplot(121)
for i in np.arange(4):
    clear_artist = plt.hist(plt.imread(clear_filenames[i])[536:1336,1204:2804,0].flatten(), 
                            color='k', bins=50, log=True, histtype='step', lw=3, alpha=.5)
for i in np.arange(9):
    cloudy_artist = plt.hist(plt.imread(cloudy_filenames[i])[536:1336,1204:2804,0].flatten(), 
                            color='r', bins=50, log=True, histtype='step', lw=3, alpha=.5)
plt.ylabel('Pixel counts')
plt.xlabel('Pixel values')
plt.legend((clear_artist[-1][0], cloudy_artist[-1][0]), ('Clear', 'Cloudy'))

#plot some global statistics
plt.subplot(122)
for i in np.arange(5):
    im_clear  = plt.imread(clear_filenames[i])[536:1336,1204:2804,0]
    cal = np.resize(np.median(plt.imread(clear_filenames[i])[536:1336,0:500,0],axis=1),(1600,800)).T
    clear_vals  = (im_clear-cal).flatten()
    clear_artist = plt.scatter(np.median(clear_vals), np.std(clear_vals), c='k', s=20)
        
for i in np.arange(10):
    im_cloudy = plt.imread(cloudy_filenames[i])[536:1336,1204:2804,0]
    cloudy_vals  = im_cloudy.flatten()
    cloudy_artist = plt.scatter(np.median(cloudy_vals), np.std(cloudy_vals), c='r', s=20)
plt.legend((clear_artist, cloudy_artist), ('Clear', 'Cloudy'))
plt.xlabel('Median pixel value')
plt.ylabel('Standard deviation of pixel values')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Text(0,0.5,'Standard deviation of pixel values')

In this clear fields the typical pixel value is around 12 with a long tail - pixels with stars - towards saturation at 255. The cloudy images have a wider distribution and much more variation in median value, which can be equal to or greater than that typical of clear fields.

Since we eventually want to classify small areas on the sky we should look at subimages as well, using 80x80 pixel divisions. As a third descriptor for these subimages we compute the entropy, a measure of the statistical randomness. It is calculated from the normalized histogram $h$ as $-\Sigma\ h\times\ln(h)$

In [14]:
from numpy.lib.stride_tricks import as_strided 
    
def make_subims(im, size=80):
    '''Segment image into subimages of size x size as a size x size x n array.'''
    nsubim, rem = divmod(im.shape[0]*im.shape[1], size**2)
    assert rem == 0, "Image cannot be evenly divided that way"
    subims = as_strided(im, shape=(im.shape[0], im.shape[1], size, size), 
                        strides=im.strides+im.strides)[::size,::size].reshape(nsubim,size,size)
    return subims
    
def entropy(subim):
    '''Comput the Shannon entropy of an image'''
    p = np.histogram(subim.flatten(),bins=np.linspace(0,255,256), density=True)[0]
    return -sum(p[p>0]*np.log(p[p>0]))

# # # # # # # # # # # # # # # # # # # # # # # #
#plot some local statistics - divide into 80x80 patches
plt.figure()
for i in np.arange(5):
    #clear
    im  = plt.imread(clear_filenames[i])[:,:,0]
    cal = np.resize(np.median(im[:,0:500],axis=1),(4008,2672)).T
    im = (im-cal)[536:1336,1204:2804]
    #trick to get subimages
    subim  = as_strided(im, shape=(800, 1600, 80, 80), strides=im.strides+im.strides)[::80,::80].reshape(200,80,80)
    entropyarr = np.zeros(len(subim))
    for j in np.arange(len(subim)):
        entropyarr[j] = entropy(subim[j])

    plt.subplot(221)
    clear_artist = plt.scatter(np.median(subim, axis=(1,2)), entropyarr, c='k', alpha=0.4, s=10, edgecolor='none')
    plt.ylabel('Entropy')
    plt.subplot(223)
    plt.scatter(np.median(subim, axis=(1,2)), np.std(subim, axis=(1,2)), c='k', alpha=0.4, s=10, edgecolor='none')
    plt.ylabel('Standard deviation')
    plt.xlabel('Median')
    plt.subplot(224)
    plt.scatter(entropyarr, np.std(subim, axis=(1,2)), c='k', alpha=0.4, s=10, edgecolor='none')
    plt.xlabel('Entropy')

for i in np.arange(10):
    #cloudy
    im  = plt.imread(cloudy_filenames[i])[:,:,0]
    cal = np.resize(np.median(im[:,0:500],axis=1),(4008,2672)).T
    im = (im-cal)[536:1336,1204:2804]
    #trick to get subimages
    subim  = as_strided(im, shape=(800, 1600, 80, 80), strides=im.strides+im.strides)[::80,::80].reshape(200,80,80)
    entropyarr = np.zeros(len(subim))
    for j in np.arange(len(subim)):
        entropyarr[j] = entropy(subim[j])
        
    plt.subplot(221)
    cloudy_artist = plt.scatter(np.median(subim, axis=(1,2)), entropyarr, c='r', alpha=0.4, s=10, edgecolor='none')
    plt.ylabel('Entropy')
    plt.subplot(223)
    plt.scatter(np.median(subim, axis=(1,2)), np.std(subim, axis=(1,2)), c='r', alpha=0.4, s=10, edgecolor='none')
    plt.ylabel('Standard deviation')
    plt.xlabel('Median')
    plt.subplot(224)
    plt.scatter(entropyarr, np.std(subim, axis=(1,2)), c='r', alpha=0.4, s=10, edgecolor='none')
    plt.xlabel('Entropy')
plt.legend((clear_artist, cloudy_artist), ('Clear', 'Cloudy'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

/Users/hendel/anaconda2/envs/py37/lib/python3.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


Individual patches are also nicely distinct in median - standard deviation - entropy space.

Let's try SVM to create a classifier. This has the advantage vs. e.g. k-nearest neightbor or k-means that all the data doens't have to be carried around later, we can just use the hypersurface if desired.

In [34]:
from sklearn.svm import SVC

#Compute the features for each class
entropyarr = np.zeros((15,200))
stdarr = np.zeros((15,200))
medianarr = np.zeros((15,200))
classesarr = np.zeros((15,200))
for i in np.arange(5):
    im  = plt.imread(clear_filenames[i])[:,:,0]
    cal = np.resize(np.median(im[:,0:500],axis=1),(4008,2672)).T
    im = (im-cal)
    im = im[536:1336,1204:2804]
    subim  = as_strided(im, shape=(800, 1600, 80, 80), strides=im.strides+im.strides)[::80,::80].reshape(200,80,80)
    for j in np.arange(len(subim)):
        entropyarr[i][j] = entropy(subim[j])
    stdarr[i] = np.std(subim, axis=(1,2))
    medianarr[i] = np.median(subim, axis=(1,2))
    classesarr[i] = np.zeros((200))
for i in np.arange(10):
    im  = plt.imread(cloudy_filenames[i])[:,:,0]
    cal = np.resize(np.median(im[:,0:500],axis=1),(4008,2672)).T
    #im = (im-cal)
    im=im[536:1336,1204:2804]
    subim  = as_strided(im, shape=(800, 1600, 80, 80), strides=im.strides+im.strides)[::80,::80].reshape(200,80,80)
    for j in np.arange(len(subim)):
        entropyarr[i+5][j] = entropy(subim[j])
    stdarr[i+5] = np.std(subim, axis=(1,2))
    medianarr[i+5] = np.median(subim, axis=(1,2))
    classesarr[i+5] = np.ones((200))

#reshape
features = np.array([medianarr.flatten(), stdarr.flatten(),entropyarr.flatten()]).T
classes = classesarr.flatten().T

svm = SVC(gamma='auto')
#double weight clear since we didn't uses as many
weights = -(classes-2)
svm.fit(features, classes, sample_weight=weights)

#given the 3000 inputs, how many are predicted correctly?
sum(svm.predict(features)==classes)


2920

The classifier seems to work correctly - skipping cross-validation etc. for time

How about one of the mixed clear/cloudy images that we area actually interested in?

In [35]:
mixed_filenames = ['2020-04-05/hires_05h52m59s_2020-04-06_EDT.jpg','2020-04-07/hires_00h27m23s_2020-04-08_EDT.jpg']
mim = plt.imread(mixed_filenames[0])[350:1950,1300:2100,0]
#mim = plt.imread(mixed_filenames[1])[536:1336,1204:2804]
plt.figure()
plt.imshow(mim,vmax=70,cmap='gray')


# # # # # # # # # # # # # # # # # # # # # # # #
#plot some local statistics - divide into 80x80 patches
plt.figure()
for i in np.arange(5):
    #clear
    im  = plt.imread(clear_filenames[i])[:,:,0]
    cal = np.resize(np.median(im[:,0:500],axis=1),(4008,2672)).T
    im = (im-cal)[536:1336,1204:2804]
    #trick to get subimages
    subim  = as_strided(im, shape=(800, 1600, 80, 80), strides=im.strides+im.strides)[::80,::80].reshape(200,80,80)
    entropyarr = np.zeros(len(subim))
    for j in np.arange(len(subim)):
        entropyarr[j] = entropy(subim[j])

    plt.subplot(221)
    clear_artist = plt.scatter(np.median(subim, axis=(1,2)), entropyarr, c='k', alpha=0.4, s=10, edgecolor='none')
    plt.ylabel('Entropy')
    plt.subplot(223)
    plt.scatter(np.median(subim, axis=(1,2)), np.std(subim, axis=(1,2)), c='k', alpha=0.4, s=10, edgecolor='none')
    plt.ylabel('Standard deviation')
    plt.xlabel('Median')
    plt.subplot(224)
    plt.scatter(entropyarr, np.std(subim, axis=(1,2)), c='k', alpha=0.4, s=10, edgecolor='none')
    plt.xlabel('Entropy')

for i in np.arange(10):
    #cloudy
    im  = plt.imread(cloudy_filenames[i])[:,:,0]
    cal = np.resize(np.median(im[:,0:500],axis=1),(4008,2672)).T
    im = (im-cal)[536:1336,1204:2804]
    #trick to get subimages
    subim  = as_strided(im, shape=(800, 1600, 80, 80), strides=im.strides+im.strides)[::80,::80].reshape(200,80,80)
    entropyarr = np.zeros(len(subim))
    for j in np.arange(len(subim)):
        entropyarr[j] = entropy(subim[j])
        
    plt.subplot(221)
    cloudy_artist = plt.scatter(np.median(subim, axis=(1,2)), entropyarr, c='r', alpha=0.4, s=10, edgecolor='none')
    plt.ylabel('Entropy')
    plt.subplot(223)
    plt.scatter(np.median(subim, axis=(1,2)), np.std(subim, axis=(1,2)), c='r', alpha=0.4, s=10, edgecolor='none')
    plt.ylabel('Standard deviation')
    plt.xlabel('Median')
    plt.subplot(224)
    plt.scatter(entropyarr, np.std(subim, axis=(1,2)), c='r', alpha=0.4, s=10, edgecolor='none')
    plt.xlabel('Entropy')
    
#mixed
im  = plt.imread('2020-04-05/hires_05h52m59s_2020-04-06_EDT.jpg')[:,:,0]
cal = np.resize(np.median(im[:,0:500],axis=1),(4008,2672)).T
#im = (im-cal)
im = im[536:2136,1204:2004]
subim  = as_strided(im, shape=(1600, 800, 80, 80), strides=im.strides+im.strides)[::80,::80].reshape(200,80,80)


#im = plt.imread(mixed_filenames[0])[:,:,0]
#cal = np.resize(np.median(im[:,0:500],axis=1),(4008,2672)).T
##im = im-cal
#im = im[536:1336,1204:2804]
#subim  = as_strided(im, shape=(800, 1600, 80, 80), strides=im.strides+im.strides)[::80,::80].reshape(200,80,80)


entropyarr = np.zeros(len(subim))
for j in np.arange(len(subim)):
    entropyarr[j] = entropy(subim[j])

plt.subplot(221)
mixed_artist = plt.scatter(np.median(subim, axis=(1,2)), entropyarr, c='b', alpha=1, s=10, edgecolor='none')
plt.ylabel('Entropy')
plt.subplot(223)
plt.scatter(np.median(subim, axis=(1,2)), np.std(subim, axis=(1,2)), c='b', alpha=1, s=10, edgecolor='none')
plt.ylabel('Standard deviation')
plt.xlabel('Median')
plt.subplot(224)
plt.scatter(entropyarr, np.std(subim, axis=(1,2)), c='b', alpha=1, s=10, edgecolor='none')
plt.xlabel('Entropy')


plt.legend((clear_artist, cloudy_artist, mixed_artist), ('Clear', 'Cloudy', 'Mixed'))

/Users/hendel/anaconda2/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

/Users/hendel/anaconda2/envs/py37/lib/python3.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [36]:
entropyarr = np.zeros((200))
for j in np.arange(len(subim)):
    entropyarr[j] = entropy(subim[j])
stdarr = np.std(subim, axis=(1,2))
medianarr = np.median(subim, axis=(1,2))

mfeatures = np.array([medianarr.flatten(), stdarr.flatten(),entropyarr.flatten()]).T

mclasses = svm.predict(mfeatures)
mclassim = mclasses.reshape(20,10)
plt.figure()
plt.subplot(121)
plt.imshow(mim,vmax=30,vmin=0,cmap='gray')
plt.subplot(122)
plt.imshow(mclassim, vmin=0, vmax=1, cmap='gray')


/Users/hendel/anaconda2/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

## Conclusion

There are definitley some inter-frame calibration issues remaining but clearly the above shows that the more-clear parts of the image are indeed classified mostly as being clear - a good start!

To-do:
- Write a wrapper to take a target list of (RA, Dec)s, do the transformation from section 1, then ask if it is in a clear part of the image using the SVM from section 2
- Get observatory to upload frames more often!
 - Necessary to eventually predict the future; clouds move too far in 5m intervals
- Improve image calibration